# Project Overview

Splunk has been deployed to collect Oracle database activity. The deployment is located in a closed network with no access to the internet. Because of this, we need to create a list of the error codes that can be referenced offline to provide additonal context to the logs being collected. The site "http://www.ora-code.com", is known to contain the information needed. We need to collect a copy of all possible Oracle error codes and their desctiptions. We will use this data to perfom correlation in Splunk. 

## Requirements
Extract the contents of ora-code.com and convert the content to a format that can loaded into Splunk and used as a refrence source.


### Tools

For this project we will be using the following tools. 
* Language
  * Python 
  * Python Modules
    * Scrapy
    * re (Regular Expressions)


## Order of Operations

 
* Identify design patterns 
* Define elements that contain data we need to collect
* on which we can base the scraper's algorithim.
* Test and re-factor as nessary to resolve any issues
* Validate a sample of the scraped data
* Run scraper


### Identify Design Patterns

On the ORA-Code site the details for each error code is hosted on its own subdomain page. Each subdomain name follows the standard Oracle error code naming convention ORA-[return code]. 

Each of the detail pages contain a table with the 3 important pieces of data. The error message, the cause and a suggested action to take. This is the information that will be extracted from each detail page.

Links to each of the detail pages can be found on the "www" subdomain. The links are spread across several numbered pages in groups of about 100 on each page.  The URLs for each page use the following format, *www.ora-code.com/code-#.html*, where "#" represents the page number. 

At the bottom of each link summary page, are indexed links to other summary pages. The index contains links for up to 20 summary pages at a time. The displayed portion of the index of a given summary page will contain links to the 10 preceeding and 10 successive pages in the page sequence.

* www
  * /code-#.html
    * [Detail Page Links]
    * [Summary Page Links]

* ORA-#####
    * [Error Message]
    * [Cause]
    * [Action]

### Define Elements
Scrapy can use either CSS selectors oe Xpaths. I prefer to use CSS Selectors. To make sure we're using teh right CSS selectors we will leverage a scrapy shell, which is an interactive shell that will let us see teh data that will be returned. 

**Open a scrapy shell  (Same commands in BASH, CMD, and Powershell):**
```powershell 
> scrapy shell "http://www.ora-code.com"



```

### Sudo Algorithim 
Now that we've identified a pattern and understand how the site is organized. We need to come up with a systematic way of spidering through it.

Today I'm only interested in the error coded that being in "ORA" 
If you click on each of the error message codes, you will see there is an **Error message**, **Cause** and **Action**, for each of the error codes.


In [2]:
import scrapy
scrapy shell "http://www.ora-code.com"

SyntaxError: invalid syntax (<ipython-input-2-5171e4cfc80b>, line 2)

## Building The Scraper

During the initial development of a script, I personally do not worry about clean code, I'm just looking a working script as quickly as possible. Once I have a working prototype, then I refine my code. You do not have to take this approach, this is just how I do it, usually because I'm trying to fix a real world issue, and the script is just a tool to help me complete teh taks faster. If I spend more time on writing the script than it would take to perform the task manually, then what's the point?  
 On the other hand, If I'm going to reuse the code or need to maintain it for teh future, I will go back re-factor, ad add comments so that it can be updated easily by anyone who happens to be using it. 

Since I am writing this to be helpful to other Cyber Security folks, and new scripters, I will write my script neatly. 

I want to start a new scrapy project. It is not always nessary especially for a simple scraper like this. but I plan on building this scraper out with additonal features in the future that will require some advances scrapy functions and configuration settings. 

**  Linux BASH Terminal:**
```BASH
scrapy startproject OraCode
cd OraCode\OraCode\spiders
touch OraSpider.py
```

** Windows Powershell Terminal **
```powershell
scrapy startproject OraCode
set-location OraCode\OraCode\spiders
new-item OraSpider.py
```







**OraSpider.py:**
```python
import re #search and select by regular expression
import scrapy

class OraErrors(scrapy.Spider):
    name = "OracleErrors"
    pageset = []
    codeset = []

    def start_requests(self):
        urls = ["http://www.ora-code.com/code-1.html"]
        for link in urls:
            yield scrapy.Request(url=link, callback=self.parse)

    def parse(self, response):
        basequery = response.css("table[bgcolor='#ffffff']:nth-child(1)")[2].css("a::attr(href)")
        urllist = basequery.extract()
        for url in urllist:
            if  "html" not in url:
                yield url
                if url not in self.codeset:
                    self.codeset.append(url)
            else:
                if url not in self.pageset:
                    nextpage = "http://www.ora-code.com/" + url
   
```

Sample test for regular expressions
```
<GET http://ORA-00028.ora-code.com>
<GET http://ORA-06622.ora-code.com>
<GET http://ORA-02238.ora-code.com>
```

## Final Script

```python
import re
import scrapy

class OraErrors(scrapy.Spider):
    name = "OracleErrors"
    pageset = []
    codeset = []

    def start_requests(self):
        urls = ["http://www.ora-code.com/code-1.html"
               ]
        for link in urls:
            yield scrapy.Request(url=link, callback=self.parse)

    def parse(self, response):
        basequery = response.css("table[bgcolor='#ffffff']:nth-child(1)")[2].css("a::attr(href)")
        urllist = basequery.extract()
        for url in urllist:
            if  "html" not in url:
                yield url
                if url not in self.codeset:
                    self.codeset.append(url)
                    yield scrapy.Request(url=url, callback=self.ora_extract)
            else:
                if url not in self.pageset:
                    nextpage = "http://www.ora-code.com/" + url
                    yield scrapy.Request(url=nextpage, callback=self.parse)
    
    def ora_extract(self, response):
        error = response.css("table > tr:nth-child(1)")
        ercode = re.findall(r"ORA-\d{5}", str(response.request))
        cause = response.css("table > tr:nth-child(2) > td:nth-child(2)::text").extract()
        action = response.css("table > tr:nth-child(3) > td:nth-child(2)::text").extract()
        yield{
            "errorcode" : ercode,
            "errormessage" :  error.css("td:nth-child(2)::text").extract(),
            'cause' : str(cause),
            'action': str(action),
            'reference' : re.findall(r"ORA-\d{5}\.ora-code.com", str(response.request))
        }

```


## Run Scrapy without a project 

``` Python
scrapy runspider "path/to/the/file.py" -o testoutput.json

```